In [1]:
%%writefile goodfrombad.py
#This one tries to to tell good blots from bad by particle_count`ing
# over twenty goes to one directory under twenty goes to another
from PIL import Image, ImageFilter, ImageOps
import numpy as np
import random
from random import randint
import sys
import time
import cv2
import pylab
from scipy import ndimage
count = 0
while count < 200:
    imgsize = 640, 640
    color = (0, 0, 0)
    img = Image.new("RGB", imgsize, "white")
    max_range = randint(100,700)
    for j in range(0,max_range):
        start = (random.randrange(0, imgsize[0]/2), random.randrange(0, imgsize[1]))
        point = start
        img.putpixel(point, color)
        """
        point_list = [p for p in 
                      [(point[0], point[1]+1), (point[0], point[1]-1), 
                       #(point[0]+1, point[1]+1), (point[0]-1, point[1]-1), 
                       #(point[0]-1, point[1]+1), (point[0]+1, point[1]-1), 
                       (point[0]+1, point[1]), (point[0]-1, point[1])]
                      if 0 < p[0] and 0 < p[1] < imgsize[1] and p not in avoid_points]
        """

        blotsize = random.randrange(0, 640)
        for i in range(blotsize):
            directions = [(point[0], point[1]+1), (point[0], point[1]-1),
                         (point[0]+1, point[1]+1), (point[0]-1, point[1]-1), 
                         (point[0]-1, point[1]+1), (point[0]+1, point[1]-1),                       
                         (point[0]+1, point[1]), (point[0]-1, point[1])]
            toremove = []
            for direction in directions:
                if direction[0]>=(imgsize[0]/2) or direction[1]>=imgsize[1] or direction[0]<0 or direction[1]<0:
                    toremove.append(direction)
            for d in toremove:
                directions.remove(d)
            point = random.choice(directions)
            img.putpixel(point, color)


    cropped = img.crop((0, 0, imgsize[0]//2, imgsize[1]))
    img = img.transpose(Image.FLIP_LEFT_RIGHT)
    img.paste(cropped, (0, 0, imgsize[0]//2, imgsize[1]))
    rad = randint(5,30)
    img = img.filter(ImageFilter.GaussianBlur(radius=rad))
    img.save("images/blot.png")


    def binarize_array(numpy_array, threshold=200):
        """Binarize a numpy array."""
        for i in range(len(numpy_array)):
            for j in range(len(numpy_array[0])):
                if numpy_array[i][j] > threshold:
                    numpy_array[i][j] = 255
                else:
                    numpy_array[i][j] = 0
        return numpy_array

    filename0=('images/blot.png')

    im = Image.open(filename0)
    im_grey = im.convert('LA') # convert to grayscale
    width,height = im.size

    total=0
    for i in range(0,width):
        for j in range(0,height):
            total += im_grey.getpixel((i,j))[0]

    mean = total / (width * height)

    image_file = Image.open(filename0)
    imagex = image_file.convert('L')  # convert image to monochrome
    imagey = np.array(imagex)
    #imagez = binarize_array(imagey, threshold)
    imagez = binarize_array(imagey, mean)
    time.sleep(2)
    filename = time.strftime("TEMP/tmpp.png")
    cv2.imwrite(filename, imagez)

    im = cv2.imread('TEMP/tmpp.png')
    pylab.figure(0)
    pylab.imshow(im)

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    maxValue = 255
    adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C#cv2.ADAPTIVE_THRESH_MEAN_C #cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    thresholdType = cv2.THRESH_BINARY#cv2.THRESH_BINARY #cv2.THRESH_BINARY_INV
    blockSize = 5 #odd number like 3,5,7,9,11
    C = -3 # constant to be subtracted
    im_thresholded = cv2.adaptiveThreshold(gray, maxValue, adaptiveMethod, thresholdType, blockSize, C) 
    labelarray, particle_count = ndimage.measurements.label(im_thresholded)
    
    if particle_count < 20:
        print"JUST RIGHT : ",particle_count
        filename = time.strftime("TEMP/good%Y%m%d%H%M%S.png")
        ImageOps.expand(Image.open('TEMP/tmpp.png').convert("RGB"),border=30,fill='red').save(filename)
        #cv2.imwrite(filename, imagez)
    else:
        print "TOO MUCH : ",particle_count
        filename = time.strftime("junk/toomany%Y%m%d%H%M%S.png")
        ImageOps.expand(Image.open('TEMP/tmpp.png').convert("RGB"),border=30,fill='red').save(filename)
        #cv2.imwrite(filename, imagez)
    print count    
    count = count+1




Writing goodfrombad.py


In [2]:
%%writefile GOODblot.py
import random
from PIL import Image, ImageFilter, ImageOps
import time
import cv2
from random import randint
import numpy as np
import os
#imgsize = (2000, 1000)
#seed_count = 10
#seed_max_size = 18000
count= 0
while count < 20:
    imgsize = (640, 640)
    seed_count = randint(6, 10)
    seed_max_size = randint(5000,16000)

    margin_h = 60
    margin_v = 60
    degradation = 10
    max_white = 100

    color = (0, 0, 0)
    img = Image.new("RGB", imgsize, "white")


    def next_points(point, avoid_points=[], shuffle=True):
        point_list = [p for p in 
                      [(point[0], point[1]+1), (point[0], point[1]-1), 
                       #(point[0]+1, point[1]+1), (point[0]-1, point[1]-1), 
                       #(point[0]-1, point[1]+1), (point[0]+1, point[1]-1), 
                       (point[0]+1, point[1]), (point[0]-1, point[1])]
                      if 0 < p[0] and 0 < p[1] < imgsize[1] and p not in avoid_points]

        for idx in range(len(point_list)):
            if point_list[idx][0] > imgsize[0]//2:
                point_list[idx] = (point[0], 
                                   point_list[idx][1] if point_list[idx][1] != point[1] else random.choice([point[1]+1,
                                                                                                            point[1]-1]))

        point_list = [p for p in point_list                  
                      if 0 < p[0] and 0 < p[1] < imgsize[1] and p not in avoid_points]

        if shuffle:
            random.shuffle(point_list)

        return point_list

    def degrade_color(color):
        return (color[0] + degradation, 
                color[1] + degradation,
                color[2] + degradation)

    def upgrade_color(color):
        return (color[0] - degradation//2, 
                color[1] - degradation//2,
                color[2] - degradation//2)

    def spread(img, point, color):
        if color[0] <= max_white and img.getpixel(point)[0] > color[0]:
            img.putpixel(point, color)
            points = next_points(point, shuffle=False)
            color = degrade_color(color)
            for point in points:
                spread(img, point, color)

    old_points = []
    posible_root_points = []
    for seed in range(0, seed_count):
        #print("Seed: %d" % seed)
        point = None
        while not point or point in old_points:
            point = (random.randrange(0 + margin_h, imgsize[0]//2), 
                     random.randrange(0 + margin_v, imgsize[1] - margin_v))
        old_points.append(point)
        posible_root_points.append(point)
        img.putpixel(point, color)

        seedsize = random.randrange(0, seed_max_size)
        #print("Seed size: %d" % seedsize)
        flow = 0
        for progress in range(0, seedsize):
            flow += 1
            points = next_points(point, old_points)
            try:
                point = points.pop()
            except IndexError:
                posible_root_points.remove(point)
                #print("Looking for old points... Seed: %d Seed Size: %d "
                      "Progress: %d Flow: %d Statistic: %d" % (seed,
                                                               seedsize,
                                                               progress,
                                                               flow, 
                                                               len(posible_root_points)))
                for idx in reversed(range(0, len(posible_root_points))):
                    points = next_points(posible_root_points[idx], old_points)
                    try:
                        point = points.pop()
                        #print("Using old point...")
                        flow = 0
                        break;
                    except IndexError:
                        posible_root_points.pop()
                if not point:
                    #print("No way!")
                    break

            old_points.append(point)
            posible_root_points.append(point)
            img.putpixel(point, color)

            for surr_point in points:
                spread(img, surr_point, degrade_color(color))

    print ("Cropping...")
    cropped = img.crop((0, 0, imgsize[0]//2, imgsize[1]))
    img = img.transpose(Image.FLIP_LEFT_RIGHT)
    img.paste(cropped, (0, 0, imgsize[0]//2, imgsize[1]))
    img = img.filter(ImageFilter.GaussianBlur(radius=10))
    img.save("images/blot.png")


    def binarize_array(numpy_array, threshold=200):
        """Binarize a numpy array."""
        for i in range(len(numpy_array)):
            for j in range(len(numpy_array[0])):
                if numpy_array[i][j] > threshold:
                    numpy_array[i][j] = 255
                else:
                    numpy_array[i][j] = 0
        return numpy_array

    filename0=('images/blot.png')

    im = Image.open(filename0)
    im_grey = im.convert('LA') # convert to grayscale
    width,height = im.size

    total=0
    for i in range(0,width):
        for j in range(0,height):
            total += im_grey.getpixel((i,j))[0]

    mean = total / (width * height)

    image_file = Image.open(filename0)
    imagex = image_file.convert('L')  # convert image to monochrome
    imagey = np.array(imagex)
    #imagez = binarize_array(imagey, threshold)
    imagez = binarize_array(imagey, mean)
    time.sleep(2)
    filename = time.strftime("blot/tmmmp.png")
    cv2.imwrite(filename, imagez)
    filename = time.strftime("blot/GOODblots%Y%m%d%H%M%S.png")
    ImageOps.expand(Image.open('blot/tmmmp.png').convert("RGB"),border=30,fill='red').save(filename)
    print "GoodBlot : ",count
    count=count+1    

Writing GOODblot.py


In [3]:
!mkdir blot

In [4]:
from PIL import Image, ImageFilter, ImageOps
import numpy as np
import random
from random import randint
import sys
import time
import cv2
count = 0
while count<20:
    imgsize = 640, 640
    color = (0, 0, 0)
    img = Image.new("RGB", imgsize, "white")
    max_range = randint(100,700)
    for j in range(0,max_range):
        start = (random.randrange(0, imgsize[0]/2), random.randrange(0, imgsize[1]))
        point = start
        img.putpixel(point, color)
        """
        point_list = [p for p in 
                      [(point[0], point[1]+1), (point[0], point[1]-1), 
                       #(point[0]+1, point[1]+1), (point[0]-1, point[1]-1), 
                       #(point[0]-1, point[1]+1), (point[0]+1, point[1]-1), 
                       (point[0]+1, point[1]), (point[0]-1, point[1])]
                      if 0 < p[0] and 0 < p[1] < imgsize[1] and p not in avoid_points]
        """

        #blotsize = random.randrange(0, 640)
        blotsize = random.randrange(0, max_range)
        for i in range(blotsize):
            directions = [(point[0], point[1]+1), (point[0], point[1]-1),
                         (point[0]+1, point[1]+1), (point[0]-1, point[1]-1), 
                         (point[0]-1, point[1]+1), (point[0]+1, point[1]-1),                       
                         (point[0]+1, point[1]), (point[0]-1, point[1])]
            toremove = []
            for direction in directions:
                if direction[0]>=(imgsize[0]/2) or direction[1]>=imgsize[1] or direction[0]<0 or direction[1]<0:
                    toremove.append(direction)
            for d in toremove:
                directions.remove(d)
            point = random.choice(directions)
            img.putpixel(point, color)


    cropped = img.crop((0, 0, imgsize[0]//2, imgsize[1]))
    img = img.transpose(Image.FLIP_LEFT_RIGHT)
    img.paste(cropped, (0, 0, imgsize[0]//2, imgsize[1]))
    rad = randint(5,30)
    img = img.filter(ImageFilter.GaussianBlur(radius=rad))
    img.save("blots/blot.png")


    def binarize_array(numpy_array, threshold=200):
        """Binarize a numpy array."""
        for i in range(len(numpy_array)):
            for j in range(len(numpy_array[0])):
                if numpy_array[i][j] > threshold:
                    numpy_array[i][j] = 255
                else:
                    numpy_array[i][j] = 0
        return numpy_array

    filename0=('blots/blot.png')

    im = Image.open(filename0)
    im_grey = im.convert('LA') # convert to grayscale
    width,height = im.size

    total=0
    for i in range(0,width):
        for j in range(0,height):
            total += im_grey.getpixel((i,j))[0]

    mean = total / (width * height)

    image_file = Image.open(filename0)
    imagex = image_file.convert('L')  # convert image to monochrome
    imagey = np.array(imagex)
    #imagez = binarize_array(imagey, threshold)
    imagez = binarize_array(imagey, mean)
    time.sleep(2)
    filename = time.strftime("blots/tmpp.png")
    cv2.imwrite(filename, imagez)

    filename = time.strftime("blots/blot%Y%m%d%H%M%S.png")
    ImageOps.expand(Image.open('blots/tmpp.png').convert("RGB"),border=30,fill='red').save(filename)

    print count    
    count = count+1



SyntaxError: Missing parentheses in call to 'print'. Did you mean print(count)? (2084551675.py, line 87)

In [5]:
*python GOODblot.py

SyntaxError: invalid syntax (1629887142.py, line 1)

# Blot Fixer - will repair the badblots

In [6]:
from PIL import Image, ImageChops
import time
import os
import random
count = 0    
while count <3:    
    path = r"blots/"
    base_image = random.choice([
        x for x in os.listdir(path)
        if os.path.isfile(os.path.join(path, x))
        ])
    filename0=(path+base_image)

    path0 = r"junk/"
    base_image0 = random.choice([
        x0 for x0 in os.listdir(path0)
        if os.path.isfile(os.path.join(path0, x0))
        ])
    filename00=(path0+base_image0)

    path1 = r"blots/"
    base_image1 = random.choice([
        x1 for x1 in os.listdir(path1)
        if os.path.isfile(os.path.join(path1, x1))
        ])
    mask0=(path1+base_image1)

    im0 = Image.open(filename0)
    im1 = im0.resize((700,700), Image.NEAREST)

    im01 = Image.open(filename00)
    im2 = im01.resize((700,700), Image.NEAREST)

    im03 = Image.open(mask0).convert("RGBA")
    
    result1 = ImageChops.composite(im1, im2, im03)
    #result = result1.resize((640,640), Image.NEAREST)
    #filename = time.strftime("tmp/blotstuff.jpg")
    
    
    filename = time.strftime("blots/fixedblot%Y%m%d%H%M%S.png")
    result1.save(filename)
    
    
    #ImageOps.expand(Image.open('tmp/blotstuff.jpg').convert("RGB"),border=30,fill='red').save(filename)
    time.sleep(4)
    count = count +1
   

FileNotFoundError: [Errno 2] No such file or directory: 'blots/'

# NICE 

In [7]:
# Good stuff
# Makes a mask from any image
# It takes the mean of a image and uses it as a threshold
from PIL import Image
import time
import random
from random import randint 
import cv2
import numpy as np
import os

def binarize_array(numpy_array, threshold=150):
    """Binarize a numpy array."""
    for i in range(len(numpy_array)):
        for j in range(len(numpy_array[0])):
            if numpy_array[i][j] > threshold:
                numpy_array[i][j] = 255
            else:
                numpy_array[i][j] = 0
    return numpy_array

filename0=('fix/bot.png')
im = Image.open(filename0)
im = im.convert('LA')
img = im.filter(ImageFilter.GaussianBlur(radius=5))
cropped = img.crop((0, 0, imgsize[0]//2, imgsize[1]))
img = img.transpose(Image.FLIP_LEFT_RIGHT)
img.paste(cropped, (0, 0, imgsize[0]//2, imgsize[1]))


img.save("junk/blot.png")

    
image_file = Image.open("junk/blot.png")

width,height = image_file.size
total=0
for i in range(0,width):
    for j in range(0,height):
        total += im_grey.getpixel((i,j))[0]
mean = total / (width * height)

imagex = image_file.convert('L')  # convert image to monochrome
imagey = np.array(imagex)
#imagez = binarize_array(imagey, threshold)
imagez = binarize_array(imagey, mean)
time.sleep(2)
cv2.imwrite("tmp/fixtmpp.png", imagez)
filename = time.strftime("fix/Manual%Y%m%d%H%M%S.png")
ImageOps.expand(Image.open('tmp/fixtmpp.png').convert("RGB"),border=30,fill='red').save(filename)

print filename

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(filename)? (665348080.py, line 52)

In [8]:
# Good stuff
# Makes a mask from any image
# It takes the mean of a image and uses it as a threshold
from PIL import Image, ImageOps
import time
import random
from random import randint 
import cv2
import numpy as np
import os

def binarize_array(numpy_array, threshold=100):
    """Binarize a numpy array."""
    for i in range(len(numpy_array)):
        for j in range(len(numpy_array[0])):
            if numpy_array[i][j] > threshold:
                numpy_array[i][j] = 255
            else:
                numpy_array[i][j] = 0
    return numpy_array

path = r"crawler1/"
base_image = random.choice([
        x for x in os.listdir(path)
        if os.path.isfile(os.path.join(path, x))
        ])
filename0=(path+base_image)

im = Image.open(filename0)
im = im.convert('LA')

img = im.filter(ImageFilter.GaussianBlur(radius=5))
cropped = img.crop((0, 0, imgsize[0]//2, imgsize[1]))
img = img.transpose(Image.FLIP_LEFT_RIGHT)
img.paste(cropped, (0, 0, imgsize[0]//2, imgsize[1]))



img.save("junk/blot.png")

    
image_file = Image.open("junk/blot.png").convert("RGB")
img = ImageOps.invert(image_file)



width,height = image_file.size
total=0
for i in range(0,width):
    for j in range(0,height):
        total += im_grey.getpixel((i,j))[0]
mean = total / (width * height)

imagex = image_file.convert('L')  # convert image to monochrome
imagey = np.array(imagex)
#imagez = binarize_array(imagey, threshold)
imagez = binarize_array(imagey, mean)
time.sleep(2)
cv2.imwrite("tmp/fixtmpp.png", imagez)
filename = time.strftime("fix/Manual%Y%m%d%H%M%S.png")
ImageOps.expand(Image.open('tmp/fixtmpp.png').convert("RGB"),border=30,fill='red').save(filename)

print filename


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(filename)? (3098220168.py, line 63)

In [9]:
# Good stuff
# Makes a mask from any image
# It takes the mean of a image and uses it as a threshold
from PIL import Image, ImageOps
import time
import random
from random import randint 
import cv2
import numpy as np
import os
count=0
while count<600:
    def binarize_array(numpy_array, threshold=100):
        """Binarize a numpy array."""
        for i in range(len(numpy_array)):
            for j in range(len(numpy_array[0])):
                if numpy_array[i][j] > threshold:
                    numpy_array[i][j] = 255
                else:
                    numpy_array[i][j] = 0
        return numpy_array
    #path = r"crawler1/"
    path = r"newstuff2/"
    base_image = random.choice([
            x for x in os.listdir(path)
            if os.path.isfile(os.path.join(path, x))
            ])
    filename0=(path+base_image)

    im = Image.open(filename0)
    im = im.convert('LA')

    img = im.filter(ImageFilter.GaussianBlur(radius=5))
    cropped = img.crop((0, 0, imgsize[0]//2, imgsize[1]))
    img = img.transpose(Image.FLIP_LEFT_RIGHT)
    img.paste(cropped, (0, 0, imgsize[0]//2, imgsize[1]))
    img.save("junk/blot.png")
    image_file = Image.open("junk/blot.png").convert("RGB")
    img = ImageOps.invert(image_file)
    width,height = image_file.size
    total=0
    for i in range(0,width):
        for j in range(0,height):
            total += im_grey.getpixel((i,j))[0]
    mean = total / (width * height)
    imagex = image_file.convert('L')  # convert image to monochrome
    imagey = np.array(imagex)
    #imagez = binarize_array(imagey, threshold)
    imagez = binarize_array(imagey, mean)
    time.sleep(2)
    cv2.imwrite("tmp/fixtmpp.png", imagez)
    filename = time.strftime("fix/Manual%Y%m%d%H%M%S.png")
    ImageOps.expand(Image.open('tmp/fixtmpp.png').convert("RGB"),border=30,fill='red').save(filename)
    count=count+1


FileNotFoundError: [Errno 2] No such file or directory: 'newstuff2/'

In [10]:
!showme fix/Manual20170903160308.png


/bin/bash: showme: command not found


# backup and experimental

In [11]:
!mkdir fix

In [13]:
import random
from PIL import Image, ImageFilter
import time
import cv2
import numpy as np
import os
#imgsize = (2000, 1000)
#seed_count = 10
#seed_max_size = 18000
imgsize = (640, 640)
seed_count = 8
seed_max_size = 9000
margin_h = 20
margin_v = 20
degradation = 10
max_white = 100
color = (0, 0, 0)
img = Image.new("RGB", imgsize, "white")
def next_points(point, avoid_points=[], shuffle=True):
    point_list = [p for p in 
                  [(point[0], point[1]+1), (point[0], point[1]-1), 
                   #(point[0]+1, point[1]+1), (point[0]-1, point[1]-1), 
                   #(point[0]-1, point[1]+1), (point[0]+1, point[1]-1), 
                   (point[0]+1, point[1]), (point[0]-1, point[1])]
                  if 0 < p[0] and 0 < p[1] < imgsize[1] and p not in avoid_points]
                   
    for idx in range(len(point_list)):
        if point_list[idx][0] > imgsize[0]//2:
            point_list[idx] = (point[0], 
                               point_list[idx][1] if point_list[idx][1] != point[1] else random.choice([point[1]+1,
                                                                                                        point[1]-1]))
                                                                                                        
    point_list = [p for p in point_list                  
                  if 0 < p[0] and 0 < p[1] < imgsize[1] and p not in avoid_points]
        
    if shuffle:
        random.shuffle(point_list)
                 
    return point_list

def degrade_color(color):
    return (color[0] + degradation, 
            color[1] + degradation,
            color[2] + degradation)
            
def upgrade_color(color):
    return (color[0] - degradation//2, 
            color[1] - degradation//2,
            color[2] - degradation//2)
            
def spread(img, point, color):
    if color[0] <= max_white and img.getpixel(point)[0] > color[0]:
        img.putpixel(point, color)
        points = next_points(point, shuffle=False)
        color = degrade_color(color)
        for point in points:
            spread(img, point, color)
            
old_points = []
posible_root_points = []
for seed in range(0, seed_count):
    print("Seed: %d" % seed)
    point = None
    while not point or point in old_points:
        point = (random.randrange(0 + margin_h, imgsize[0]//2), 
                 random.randrange(0 + margin_v, imgsize[1] - margin_v))
    old_points.append(point)
    posible_root_points.append(point)
    img.putpixel(point, color)

    seedsize = random.randrange(0, seed_max_size)
    print("Seed size: %d" % seedsize)
    flow = 0
    for progress in range(0, seedsize):
        flow += 1
        points = next_points(point, old_points)
        try:
            point = points.pop()
        except IndexError:
            posible_root_points.remove(point)
            print("Looking for old points... Seed: %d Seed Size: %d "
                  "Progress: %d Flow: %d Statistic: %d" % (seed,
                                                           seedsize,
                                                           progress,
                                                           flow, 
                                                           len(posible_root_points)))
            for idx in reversed(range(0, len(posible_root_points))):
                points = next_points(posible_root_points[idx], old_points)
                try:
                    point = points.pop()
                    print("Using old point...")
                    flow = 0
                    break;
                except IndexError:
                    posible_root_points.pop()
            if not point:
                print("No way!")
                break
            
        old_points.append(point)
        posible_root_points.append(point)
        img.putpixel(point, color)
        
        for surr_point in points:
            spread(img, surr_point, degrade_color(color))

print ("Cropping...")
cropped = img.crop((0, 0, imgsize[0]//2, imgsize[1]))
img = img.transpose(Image.FLIP_LEFT_RIGHT)
img.paste(cropped, (0, 0, imgsize[0]//2, imgsize[1]))
img = img.filter(ImageFilter.GaussianBlur(radius=10))
img.save("images/blot.png")

def binarize_array(numpy_array, threshold=200):
    """Binarize a numpy array."""
    for i in range(len(numpy_array)):
        for j in range(len(numpy_array[0])):
            if numpy_array[i][j] > threshold:
                numpy_array[i][j] = 255
            else:
                numpy_array[i][j] = 0
    return numpy_array

filename0=('images/blot.png')
    
im = Image.open(filename0)
im_grey = im.convert('LA') # convert to grayscale
width,height = im.size

total=0
for i in range(0,width):
    for j in range(0,height):
        total += im_grey.getpixel((i,j))[0]

mean = total / (width * height)
    
image_file = Image.open(filename0)
imagex = image_file.convert('L')  # convert image to monochrome
imagey = np.array(imagex)
#imagez = binarize_array(imagey, threshold)
imagez = binarize_array(imagey, mean)
time.sleep(2)
filename = time.strftime("images/%Y%m%d%H%M%S.png")
cv2.imwrite(filename, imagez)
print (filename)

Seed: 0
Seed size: 2415
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 109 Flow: 110 Statistic: 109
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 116 Flow: 7 Statistic: 115
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 124 Flow: 8 Statistic: 120
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 125 Flow: 1 Statistic: 120
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 141 Flow: 16 Statistic: 118
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 267 Flow: 126 Statistic: 241
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 272 Flow: 5 Statistic: 245
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 274 Flow: 2 Statistic: 244
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 277 Flow: 3 Statistic: 238
Using old point...
Looking for old p

Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2087 Flow: 80 Statistic: 1834
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2089 Flow: 2 Statistic: 1835
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2091 Flow: 2 Statistic: 1835
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2092 Flow: 1 Statistic: 1831
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2099 Flow: 7 Statistic: 1836
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2101 Flow: 2 Statistic: 1836
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2103 Flow: 2 Statistic: 1828
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2116 Flow: 13 Statistic: 1836
Using old point...
Looking for old points... Seed: 0 Seed Size: 2415 Progress: 2277 Flow: 161 Statistic: 1996
Using old point...
Looking for old points..

Looking for old points... Seed: 3 Seed Size: 7258 Progress: 383 Flow: 14 Statistic: 3845
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 405 Flow: 22 Statistic: 3866
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 452 Flow: 47 Statistic: 3912
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 494 Flow: 42 Statistic: 3951
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 574 Flow: 80 Statistic: 4028
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 582 Flow: 8 Statistic: 4034
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 584 Flow: 2 Statistic: 4032
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 587 Flow: 3 Statistic: 4027
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 588 Flow: 1 Statistic: 4026
Using old point...
Looking for old points... Seed: 

Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2242 Flow: 81 Statistic: 5358
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2250 Flow: 8 Statistic: 5364
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2285 Flow: 35 Statistic: 5386
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2286 Flow: 1 Statistic: 5385
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2291 Flow: 5 Statistic: 5385
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2332 Flow: 41 Statistic: 5420
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2350 Flow: 18 Statistic: 5437
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2463 Flow: 113 Statistic: 5549
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 2485 Flow: 22 Statistic: 5570
Using old point...
Looking for old point

Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3637 Flow: 11 Statistic: 6385
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3641 Flow: 4 Statistic: 6387
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3644 Flow: 3 Statistic: 6389
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3645 Flow: 1 Statistic: 6365
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3647 Flow: 2 Statistic: 6359
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3648 Flow: 1 Statistic: 6356
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3649 Flow: 1 Statistic: 6351
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3653 Flow: 4 Statistic: 6330
Using old point...
Looking for old points... Seed: 3 Seed Size: 7258 Progress: 3655 Flow: 2 Statistic: 6331
Using old point...
Looking for old points... S

...

Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2441 Flow: 1 Statistic: 9730
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2442 Flow: 1 Statistic: 9723
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2449 Flow: 7 Statistic: 9728
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2453 Flow: 4 Statistic: 9731
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2463 Flow: 10 Statistic: 9725
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2465 Flow: 2 Statistic: 9726
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2466 Flow: 1 Statistic: 9704
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2484 Flow: 18 Statistic: 9694
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 2486 Flow: 2 Statistic: 9691
Using old point...
Looking

Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3446 Flow: 4 Statistic: 10261
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3452 Flow: 6 Statistic: 10266
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3460 Flow: 8 Statistic: 10272
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3461 Flow: 1 Statistic: 10272
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3462 Flow: 1 Statistic: 10272
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3466 Flow: 4 Statistic: 10247
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3473 Flow: 7 Statistic: 10253
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3478 Flow: 5 Statistic: 10255
Using old point...
Looking for old points... Seed: 4 Seed Size: 4700 Progress: 3480 Flow: 2 Statistic: 10255
Using old point...


Looking for old points... Seed: 5 Seed Size: 2085 Progress: 399 Flow: 9 Statistic: 11524
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 405 Flow: 6 Statistic: 11526
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 423 Flow: 18 Statistic: 11542
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 428 Flow: 5 Statistic: 11546
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 430 Flow: 2 Statistic: 11545
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 434 Flow: 4 Statistic: 11540
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 435 Flow: 1 Statistic: 11535
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 436 Flow: 1 Statistic: 11522
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 460 Flow: 24 Statistic: 11535
Using old point...
Looking for old points... 

Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1816 Flow: 13 Statistic: 12544
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1817 Flow: 1 Statistic: 12544
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1819 Flow: 2 Statistic: 12541
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1820 Flow: 1 Statistic: 12535
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1827 Flow: 7 Statistic: 12533
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1831 Flow: 4 Statistic: 12516
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1832 Flow: 1 Statistic: 12516
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1833 Flow: 1 Statistic: 12501
Using old point...
Looking for old points... Seed: 5 Seed Size: 2085 Progress: 1834 Flow: 1 Statistic: 12489
Using old point...

FileNotFoundError: [Errno 2] No such file or directory: 'images/blot.png'